In [1]:
import tensorflow as tf


d:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
vocab_size = 10000
embedding_size = 200

# Q-A 的保存
source  = tf.placeholder(dtype=tf.int32,shape=[None,None],name='source_text')
target = tf.placeholder(dtype=tf.int32,shape=[None,None],name='target_text')

# tf.nn.dynamic_rnn 中有一个参数需要记录出当前batch的每个句子的长度。此处用于存储
source_sequence_length = tf.placeholder(dtype=tf.int32,shape=[None,],name='source_seq_length')
target_sequence_length = tf.placeholder(dtype=tf.int32,shape=[None,],name='target_seq_length')

# 保存于训练的词向量
embedding = tf.placeholder(dtype=tf.float32,shape=[vocab_size,embedding_size],name='embedding_size')
print(embedding)

Tensor("embedding_size:0", shape=(10000, 200), dtype=float32)


In [3]:
# encoder 的建立
encoder_embedding  = tf.nn.embedding_lookup(params=embedding,ids=source)

lstm_units = 128
lstm_layer = 1

def get_lstms(nums):
    return tf.contrib.rnn.LSTMCell(dtype=tf.float32,num_units=nums,initializer=tf.random_normal_initializer(mean=0.1,stddev=0.1,seed=123))
encoder_lstms = tf.contrib.rnn.MultiRNNCell([get_lstms(lstm_units) for _ in range(lstm_layer)])

encoder_outputs,encoder_final_state = tf.nn.dynamic_rnn(cell=encoder_lstms,dtype=tf.float32,inputs=encoder_embedding,sequence_length=source_sequence_length)

print(encoder_embedding)
print(encoder_lstms)
print(encoder_outputs)
print(encoder_final_state)  #c和h是一样的吗？


Tensor("embedding_lookup:0", shape=(?, ?, 200), dtype=float32)
Tensor("rnn/transpose_1:0", shape=(?, ?, 128), dtype=float32)
(LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 128) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_4:0' shape=(?, 128) dtype=float32>),)


outputs, last_states = tf.nn.dynamic_rnn(  cell=cell,  dtype=tf.float32, sequence_length=x_lengths,  inputs=x)

其中cell是RNN节点，比如tf.contrib.rnn.BasicLSTMCel，x是0-padding以后的数据，x_lengths是每个文本的长度。

decoder的建立需要注意一下点：
1.输入的是encoder的最终状态，
2.训练 training 时 decoder的输入时targets； 在infrence时 decoder的输入为上一个lstm结构运算的结构
3.training时候使用trainning helper 在做inference时使用greedyEmbeddingHelper

In [4]:
#decoder的建立
decoder_embedding = tf.nn.embedding_lookup(params=embedding,ids=target)

# 训练时候的decoder

# lstm - cell
decoder_lstms = tf.contrib.rnn.MultiRNNCell(cells=[get_lstms(lstm_units) for _ in range(lstm_layer)])
# helper
train_helper = tf.contrib.seq2seq.TrainingHelper(inputs=decoder_embedding,
                                                 sequence_length=target_sequence_length)
#output_layer
target_vocab_size = 1000
output_layer = tf.layers.Dense(units=target_vocab_size)
#decoder结构
trainning_decoder = tf.contrib.seq2seq.BasicDecoder(cell=decoder_lstms,
                                                    helper=train_helper,
                                                    initial_state=encoder_final_state,
                                                    output_layer=output_layer)
max_sequence_length = 25
final_outputs,final_state, final_sequence_lengths = tf.contrib.seq2seq.dynamic_decode(decoder=trainning_decoder,
                                                                                      impute_finished=True,
                                                                                      maximum_iterations=max_sequence_length)

print(train_helper)
print(output_layer)
print(trainning_decoder)
print(final_outputs)
print(final_state)
print(final_sequence_lengths)

BasicDecoderOutput(rnn_output=<tf.Tensor 'decoder/transpose:0' shape=(?, ?, 1000) dtype=float32>, sample_id=<tf.Tensor 'decoder/transpose_1:0' shape=(?, ?) dtype=int32>)
(LSTMStateTuple(c=<tf.Tensor 'decoder/while/Exit_4:0' shape=(?, 128) dtype=float32>, h=<tf.Tensor 'decoder/while/Exit_5:0' shape=(?, 128) dtype=float32>),)
Tensor("decoder/while/Exit_8:0", shape=(?,), dtype=int32)


参数说明：
train_helper的参数：inputs：embedding向量，lookup之后的矩阵，length：batch中输入的每个sequence的长度矩阵
decoder参数 decoder使用初始的BasicDecoder，参数：cell：decoder中的lstms序列，helper 使用的计算方式helper initial_helper是encoder的最终的输出状态，output_layer 是输出层的方式，一般是decens的网络结构，全连接的网络结构。因为输出的时候是一个向量，通过一个向量得到某个值，所以说通过一个全连接层来决定最终的输出是什么。
decoder的计算方式这里使用dynamic_decoder的计算方式,最大的迭代次数设置为句子长度的最大值，迭代的可以使用finish向量来进行，返回三个值：final_outputs, final_state, final_sequence_lengths

In [5]:
print(decoder_embedding)

Tensor("embedding_lookup_1:0", shape=(?, ?, 200), dtype=float32)


# inference时使用的decoder结构，参数可以和trainning时的decoder共享

可以共享的参数有：decoder_embed,decoder_cell,output_layer,需要重新定义decoder_helper,deocer结构以及decoder的计算方式

In [6]:
start_id = 0
end_id = 3
batch_size = 50
#start_tokens 是一个向量，end_token是一个标量
start_tokens =  tf.tile(tf.constant([start_id], dtype=tf.int32), [batch_size], name="start_tokens")
                #tf.tile(tf.constant([start_id], dtype=tf.int32), [batch_size], name="start_tokens")
print(start_tokens)
inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embedding=embedding,  # 这里的embedding 不需要做lookup
                                                            start_tokens=start_tokens,
                                                             end_token=end_id)
print(inference_helper)
inference_decoder = tf.contrib.seq2seq.BasicDecoder(cell=decoder_lstms,
                                                   helper=inference_helper,
                                                   initial_state=encoder_final_state,
                                                   output_layer=output_layer)

print(inference_decoder)
final_outputs_infer,_, _ = tf.contrib.seq2seq.dynamic_decode(decoder=inference_decoder,
                                                        impute_finished=True,
                                                        maximum_iterations=max_sequence_length)

print(start_tokens)
print(final_outputs_infer)

Tensor("start_tokens:0", shape=(50,), dtype=int32)
Tensor("start_tokens:0", shape=(50,), dtype=int32)
BasicDecoderOutput(rnn_output=<tf.Tensor 'decoder_1/transpose:0' shape=(50, ?, 1000) dtype=float32>, sample_id=<tf.Tensor 'decoder_1/transpose_1:0' shape=(50, ?) dtype=int32>)


#这里需要注意的是，inferencehelp中的embedding 并不需要做lookup

In [8]:
trainning_outputs = final_outputs
# 生成数据进行模型的训练和优化
source_length = 788 # source_length = len(source)
def get_batches(source,target,batch_size):
    for batch_i in range(len(source)):
        start_i = batch_i* batch_size

        source_batch = source[start_i:start_i+batch_size]
        target_batch = target[start_i:start_i+batch_size]

        source_sequence_length = []
        target_sequence_length = []

        for seq in source_batch:
            source_sequence_length.append(len(seq))
        for seq in rarget_batch:
            target_sequence_length.append(len(seq))

        yield source_batch,target_batch,source_sequence_length,target_sequence_length

In [ ]:
inference_outputs =  final_outputs_infer
trainin_logits = tf.identity()

In [ ]:
train_graph = tf.Graph()
epoch = 10
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    for epoch_i in range(epoch):
        